In [2]:
import torch
from __future__ import print_function
from torch import np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [3]:
def readFeatures(fileName, istorch = True):
    f = open(fileName, "r+")
    features = []

    for line in f.readlines():
        feat = line.split(",")
        feat = map(float, feat)
        features.append(feat)
    features = np.asarray(features)
    if istorch:
        features = torch.from_numpy(features)
        features = features.float()
    f.close()
    return features

In [4]:
def readLabels(fileName, onehot = False, istorch = True):
    f = open(fileName, "r+")
    labels = []
    for line in f.readlines():
        if onehot == True:
            label = np.zeros(10)
            label[(int)(line)] = 1
        else:
            label = (int)(line)
        labels.append(label)    
    labels = np.asarray(labels)
    if istorch:
        labels = torch.from_numpy(labels)
    f.close()
    return labels

In [5]:
trainFileNameFeatures = "train_set/train_set_images.txt"
trainFileNameLabels = "train_set/train_set_labels.txt"
trainFeatures = readFeatures(trainFileNameFeatures)
trainLabels = readLabels(trainFileNameLabels)

In [7]:
validFileNameFeatures = "validation-test/validation_set_images.txt"
validFileNameLabels = "validation-test/validation_set_labels.txt"
validFeatures = readFeatures(validFileNameFeatures)
validLabels = readLabels(validFileNameLabels)

In [13]:
validFeatures.size()

torch.Size([10000, 784])

In [8]:
testFileNameFeatures = "validation-test/test_set_images.txt"
testFileNameLabels = "validation-test/test_set_labels.txt"
testFeatures = readFeatures(testFileNameFeatures)
testLabels = readLabels(testFileNameLabels)

In [9]:
torch.cuda.set_device(3)
torch.cuda.current_device()

3L

# Convolutional Neural Network

In [10]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=(2,2))
        self.conv2 = nn.Conv2d(32, 64, 5, padding= (2,2))
        self.fc1 = nn.Linear(7*7*64, 1024)
        self.fc2 = nn.Linear(1024, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
convnet = ConvNet()
convnet.cuda()
print(convnet)

ConvNet (
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear (3136 -> 1024)
  (fc2): Linear (1024 -> 10)
)


In [ ]:
criterionConvNet = nn.CrossEntropyLoss()
optimizerConvNet = optim.Adam(convnet.parameters())

In [ ]:
b_size = 500
start_time = time.time()
for j in range(0, 100):
    running_loss = 0.0
    for i in range(0, int(trainFeatures.size()[0]), b_size):
        inputs = trainFeatures[i:i+b_size]
        inputs = inputs.view(-1,1,28,28)
        inputs = Variable(inputs.cuda())
        outputs = convnet(inputs)
        
        targets = Variable(trainLabels[i:i+b_size].cuda())
        optimizerConvNet.zero_grad()
        
        loss = criterionConvNet(outputs, targets)
        loss.backward()
        optimizerConvNet.step()
        
        running_loss += loss.data[0]
            
    if j%50==49:
        print("Iteration : " + str(j) + " took : " + str(time.time()-start_time))
        print('[%d, %5d] loss: %.15f' % (j+1, i+1, running_loss / 2000))
        running_loss = 0.0
        start_time = time.time()


### Convolutional neural network test result

In [ ]:
correct = 0
total = 0
for i in range(0, len(testFeatures)):
    images, labels = testFeatures[i], testLabels[i]
    outputs = convnet(Variable(images).view(-1,1,28,28).cuda())
    _, predicted = torch.max(outputs.data, 1)
    total += 1
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: ' + str(100.0 * correct / total*1.0))

# Feed Forward Single hidden Layer

In [ ]:
class OneLayerNet(nn.Module):
    def __init__(self):
        super(OneLayerNet, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)
    
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

oneNet = OneLayerNet()
oneNet.cuda()
print(oneNet)

In [ ]:
criteriononenet = nn.CrossEntropyLoss()
optimizeronenet = optim.Adam(oneNet.parameters())

In [ ]:
b_size = 1000
start_time = time.time()
for i in range(0,100):
    for j in range(0, len(trainFeatures), b_size):
        inputs = trainFeatures[j:j+b_size]
        inputs = Variable(inputs).cuda()
        labels = Variable(trainLabels[j:j+b_size]).cuda()
        optimizeronenet.zero_grad()
        outputs = oneNet(inputs)
        loss = criteriononenet(outputs, labels)
        loss.backward()
        optimizeronenet.step()

    if i%10==0:
        print("Iteration : " + str(j) + " took : " + str(time.time()-start_time))
        start_time = time.time()

In [ ]:
correct = 0
total = 0
for i in range(0, len(testFeatures)):
    images, labels = testFeatures[i].view(-1,784), testLabels[i]
    outputs = oneNet(Variable(images).cuda())
    _, predicted = torch.max(outputs.data, 1)
    total += 1
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: ' + str(100.0 * correct / total*1.0))

# Feed Forward Two Hidden Layer 

In [ ]:
class TwoLayerNet(nn.Module):
    def __init__(self):
        super(TwoLayerNet, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500,250)
        self.fc3 = nn.Linear(250, 10)
    
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

twoNet = TwoLayerNet()
twoNet.cuda()
print(twoNet)

In [ ]:
criterionTwoLayer = nn.CrossEntropyLoss()
optimizerTwoLayer = optim.Adam(twoNet.parameters())

In [ ]:
b_size = 1000
start_time = time.time()
for i in range(0,100):
    for j in range(0, len(trainFeatures), b_size):
        inputs = trainFeatures[j:j+b_size]
        inputs = Variable(inputs).cuda()
        labels = Variable(trainLabels[j:j+b_size]).cuda()
        
        optimizerTwoLayer.zero_grad()
        outputs = twoNet(inputs)
        loss = criterionTwoLayer(outputs, labels)
        loss.backward()
        optimizerTwoLayer.step()

    if i%10==0:
        print("Iteration : " + str(j) + " took : " + str(time.time()-start_time))
        start_time = time.time()

In [ ]:
correct = 0
total = 0
for i in range(0, len(testFeatures)):
    images, labels = testFeatures[i].view(-1,784), testLabels[i]
    outputs = twoNet(Variable(images).cuda())
    _, predicted = torch.max(outputs.data, 1)
    total += 1
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: ' + str(100.0 * correct / total*1.0))

# Feed Forward 5 Hidden Layer

In [ ]:
class FiveLayerNet(nn.Module):
    def __init__(self):
        super(FiveLayerNet, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 250)
        self.fc3 = nn.Linear(250, 200)
        self.fc4 = nn.Linear(200, 300)
        self.fc5 = nn.Linear(300, 150)
        self.fc6 = nn.Linear(150, 10)
    
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.sigmoid(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        x = self.fc6(x)
        return x

fiveNet = FiveLayerNet()
fiveNet.cuda()
print(fiveNet)

In [ ]:
criterionFiveLayer = nn.CrossEntropyLoss()
optimizerFiveLayer = optim.Adam(fiveNet.parameters())

In [ ]:
b_size = 1000
start_time = time.time()
for i in range(0,100):
    for j in range(0, len(trainFeatures), b_size):
        inputs = trainFeatures[j:j+b_size]
        inputs = Variable(inputs).cuda()
        labels = Variable(trainLabels[j:j+b_size]).cuda()
        
        optimizerFiveLayer.zero_grad()
        outputs = fiveNet(inputs)
        loss = criterionFiveLayer(outputs, labels)
        loss.backward()
        optimizerFiveLayer.step()

    if i%10==0:
        print("Iteration : " + str(j) + " took : " + str(time.time()-start_time))
        start_time = time.time()

In [ ]:
correct = 0
total = 0
for i in range(0, len(testFeatures)):
    images, labels = testFeatures[i].view(-1,784), testLabels[i]
    outputs = fiveNet(Variable(images).cuda())
    _, predicted = torch.max(outputs.data, 1)
    total += 1
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: ' + str(100.0 * correct / total*1.0))

# SVM

In [ ]:
trainFileNameFeatures = "train_set/train_set_images.txt"
trainFileNameLabels = "train_set/train_set_labels.txt"
trainFeatures = readFeatures(trainFileNameFeatures, istorch=False)
trainLabels = readLabels(trainFileNameLabels, onehot = False, istorch = False)

In [ ]:
validFileNameFeatures = "validation-test/validation_set_images.txt"
validFileNameLabels = "validation-test/validation_set_labels.txt"
validFeatures = readFeatures(validFileNameFeatures, istorch = False)
validLabels = readLabels(validFileNameLabels, onehot = False, istorch = False)

In [ ]:
testFileNameFeatures = "validation-test/test_set_images.txt"
testFileNameLabels = "validation-test/test_set_labels.txt"
testFeatures = readFeatures(testFileNameFeatures, istorch = False)
testLabels = readLabels(testFileNameLabels, onehot = False, istorch = False)

In [ ]:
from sklearn.svm import SVC

In [ ]:
SVMClassifier = SVC(C=1e3, tol=1e-5, kernel='rbf')
SVMClassifier.fit(trainFeatures, trainLabels)

In [ ]:
SVMClassifier.score(testFeatures, testLabels)